# MobileNetV2 Feature Extraction

Transfer Learning with Keras and Deep Learning
https://pyimagesearch.com/2019/05/20/transfer-learning-with-keras-and-deep-learning/

Keras: Feature extraction on large datasets with Deep Learning  
https://pyimagesearch.com/2019/05/27/keras-feature-extraction-on-large-datasets-with-deep-learning/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from imutils import paths

import tensorflow as tf
from tensorflow import keras 
from keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
## path to image dataset
BASE_DIR = "C:/Users/noqui/Desktop/FYP/Work_Folder/dataset/cassava"
TRAIN_DIR = "train"
TEST_DIR = "test"

## path to loading model
MODEL_PATH = "C:/Users/noqui/Desktop/FYP/Work_Folder/output/model"

## output paths
FEATURES_OUTPUT_PATH = "C:/Users/noqui/Desktop/FYP/Work_Folder/output/features"
LE_OUTPUT_PATH = "C:/Users/noqui/Desktop/FYP/Work_Folder/output/encoder"
MODEL_OUTPUT_PATH = "C:/Users/noqui/Desktop/FYP/Work_Folder/output/model"

for path in [FEATURES_OUTPUT_PATH, LE_OUTPUT_PATH, MODEL_OUTPUT_PATH]:
    if not os.path.exists(path):
        os.makedirs(path)
    
## essential parameters
BATCH_SIZE = 32
DIM = (224, 224)

## Feature Extraction

In [3]:
### Function to extract features 
# model        : Keras model for feature extractor
# output_shape : Output shape of the max-pooling layer
# model_name   : Name of the model

def extractFeatures(model, output_shape, model_name):
    le = None
    for split_type in [TRAIN_DIR, TEST_DIR]:

        # grab all image paths in the current path
        print(f"[INFO] processing {split_type} split...")
        path = f"{BASE_DIR}/{split_type}"
        imagePaths = list(paths.list_images(path)) # unlike os.listdir, this grabs all images from subfolders

        # randomly shuffle the image paths and then extract the class
        # labels from the file paths
        labels = [p.split(os.path.sep)[1] for p in imagePaths]
        allFeatures = [] # to be appended
        
        # fit the label encoder once
        if le is None:
            le = LabelEncoder()
            le.fit(labels)

        # loop over the images in batches
        for (b, i) in enumerate(range(0, len(imagePaths), BATCH_SIZE)):
            # extract the batch of images and labels, then initialize the
            # list of actual images that will be passed through the network
            # for feature extraction
            print(f"[INFO] processing batch {b+1}/{int(np.ceil(len(imagePaths) / float(BATCH_SIZE)))}")
            batchPaths = imagePaths[i:i + BATCH_SIZE]
            batchImages = []

            # loop over the images and labels in the current batch
            for imagePath in batchPaths:
                # load the input image using the Keras helper utility
                # while ensuring the image is resized
                image = load_img(imagePath, target_size = DIM) 
                image = img_to_array(image)

                # preprocess the image by 
                # (1) expanding the dimensions and
                # (2) subtracting the mean RGB pixel intensity from the ImageNet dataset
                image = np.expand_dims(image, axis = 0)
                image = preprocess_input(image)

                # add the image to the batch
                batchImages.append(image)

            # pass the images through the network and use the outputs as
            # our actual features, then reshape the features into a flattened volume
            batchImages = np.vstack(batchImages)
            features = model.predict(batchImages, batch_size = BATCH_SIZE)
            features = features.reshape((features.shape[0], output_shape))
            
            # append features
            for vec in features:
                allFeatures.append(vec)
        
        allFeatures = np.array(allFeatures)
        labels = np.array(labels)
        
        # dump features, labels, and imagePaths
        pickle.dump(allFeatures, open(f"{FEATURES_OUTPUT_PATH}/{model_name.lower()}_{split_type}_features.pkl", "wb"))
        pickle.dump(labels, open(f"{FEATURES_OUTPUT_PATH}/{model_name.lower()}_{split_type}_labels.pkl", "wb"))
        pickle.dump(imagePaths,  open(f"{FEATURES_OUTPUT_PATH}/{model_name.lower()}_{split_type}_imagepaths.pkl", "wb"))

    # serialize the label encoder to disk
    pickle.dump(le, open(f"{LE_OUTPUT_PATH}/encoder.pkl", "wb"))

In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# MobileNetV2 with imagenet as weights
mobilenetv2_imagenet_fe = MobileNetV2(weights = "imagenet", input_shape = (224, 224, 3), include_top = False)
mobilenetv2_imagenet_fe.trainable = True
mobilenetv2_imagenet_fe.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [5]:
# load MobileNetV2 with transfer learning 
mobilenetv2_pretrained = keras.models.load_model(f"{MODEL_PATH}/MobileNetV2.h5")

# remove the fully connected layers
mobilenetv2_pretrained_fe = Model(inputs = mobilenetv2_pretrained.inputs, outputs = mobilenetv2_pretrained.layers[-4].output)
mobilenetv2_pretrained_fe.trainable = True
mobilenetv2_pretrained_fe.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [6]:
# extracting features with pretrained weights
model_name = "Cassava_Dataset_MobileNetv2_pretrained"
extractFeatures(mobilenetv2_pretrained_fe, output_shape = 7 * 7 * 6, model_name = model_name)

[INFO] processing train split...
[INFO] processing batch 1/469
1/1 [==============================] - 1s 1s/step
[INFO] processing batch 2/469
1/1 [==============================] - 0s 423ms/step
[INFO] processing batch 3/469
1/1 [==============================] - 0s 439ms/step
[INFO] processing batch 4/469
1/1 [==============================] - 0s 455ms/step
[INFO] processing batch 5/469
1/1 [==============================] - 0s 440ms/step
[INFO] processing batch 6/469
1/1 [==============================] - 0s 419ms/step
[INFO] processing batch 7/469
1/1 [==============================] - 0s 394ms/step
[INFO] processing batch 8/469
1/1 [==============================] - 0s 403ms/step
[INFO] processing batch 9/469
1/1 [==============================] - 0s 396ms/step
[INFO] processing batch 10/469
1/1 [==============================] - 0s 400ms/step
[INFO] processing batch 11/469
1/1 [==============================] - 0s 422ms/step
[INFO] processing batch 12/469
1/1 [===================

1/1 [==============================] - 0s 407ms/step
[INFO] processing batch 99/469
1/1 [==============================] - 0s 402ms/step
[INFO] processing batch 100/469
1/1 [==============================] - 0s 407ms/step
[INFO] processing batch 101/469
1/1 [==============================] - 0s 393ms/step
[INFO] processing batch 102/469
1/1 [==============================] - 0s 393ms/step
[INFO] processing batch 103/469
1/1 [==============================] - 0s 412ms/step
[INFO] processing batch 104/469
1/1 [==============================] - 0s 422ms/step
[INFO] processing batch 105/469
1/1 [==============================] - 0s 395ms/step
[INFO] processing batch 106/469
1/1 [==============================] - 0s 393ms/step
[INFO] processing batch 107/469
1/1 [==============================] - 0s 389ms/step
[INFO] processing batch 108/469
1/1 [==============================] - 0s 396ms/step
[INFO] processing batch 109/469
1/1 [==============================] - 0s 396ms/step
[INFO] proces

1/1 [==============================] - 0s 395ms/step
[INFO] processing batch 291/469
1/1 [==============================] - 0s 407ms/step
[INFO] processing batch 292/469
1/1 [==============================] - 0s 402ms/step
[INFO] processing batch 293/469
1/1 [==============================] - 0s 400ms/step
[INFO] processing batch 294/469
1/1 [==============================] - 0s 406ms/step
[INFO] processing batch 295/469
1/1 [==============================] - 0s 408ms/step
[INFO] processing batch 296/469
1/1 [==============================] - 0s 421ms/step
[INFO] processing batch 297/469
1/1 [==============================] - 0s 394ms/step
[INFO] processing batch 298/469
1/1 [==============================] - 0s 402ms/step
[INFO] processing batch 299/469
1/1 [==============================] - 0s 392ms/step
[INFO] processing batch 300/469
1/1 [==============================] - 0s 399ms/step
[INFO] processing batch 301/469
1/1 [==============================] - 0s 397ms/step
[INFO] proce

1/1 [==============================] - 0s 406ms/step
[INFO] processing batch 14/201
1/1 [==============================] - 0s 434ms/step
[INFO] processing batch 15/201
1/1 [==============================] - 0s 423ms/step
[INFO] processing batch 16/201
1/1 [==============================] - 0s 412ms/step
[INFO] processing batch 17/201
1/1 [==============================] - 0s 413ms/step
[INFO] processing batch 18/201
1/1 [==============================] - 0s 393ms/step
[INFO] processing batch 19/201
1/1 [==============================] - 0s 401ms/step
[INFO] processing batch 20/201
1/1 [==============================] - 0s 405ms/step
[INFO] processing batch 21/201
1/1 [==============================] - 0s 417ms/step
[INFO] processing batch 22/201
1/1 [==============================] - 0s 400ms/step
[INFO] processing batch 23/201
1/1 [==============================] - 0s 402ms/step
[INFO] processing batch 24/201
1/1 [==============================] - 0s 389ms/step
[INFO] processing batch

## Preparing the Data for Train, Validation and Test

Loading generated image features and splitting them into train, validation and test.

The train set will be used for training and validation.  
The validation set will be used for testing.

Naming Convention

| Data Split | X         | y         | y (one-hot-encoded) |
| :--------- | :-------- | :-------- | :------------------ |
| Training   | `X_train` | `y_train` | `Y_train`           |
| Validation | `X_val`   | `y_val`   | `Y_val`             |
| Testing    | `X_test`  | `y_test`  | `Y_test`            |

In [3]:
# function to load features, labels and imagepath from the extraction
def getDataAndImagePaths(model_name, split_type):
    X = pickle.loads(open(f"{FEATURES_OUTPUT_PATH}/{model_name.lower()}_{split_type}_features.pkl", "rb").read())
    y = pickle.loads(open(f"{FEATURES_OUTPUT_PATH}/{model_name.lower()}_{split_type}_labels.pkl", "rb").read())
    imagePaths = pickle.loads(open(f"{FEATURES_OUTPUT_PATH}/{model_name.lower()}_{split_type}_imagepaths.pkl", "rb").read())
    
    return (X, y, imagePaths)

## MobileNetV2 with pretrained weights

In [4]:
model_name = "Cassava_Dataset_MobileNetv2_pretrained"

# load the data from disk
print(f"[INFO] loading training data for {model_name}...")
(data, labels, imagePaths) = getDataAndImagePaths(model_name, split_type = "train")

# splitting the data and label into train validation set
print(f"[INFO] splitting the datasets into train and validation...")
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size = 0.3, random_state = 42)

# load testing data from disk
print(f"[INFO] loading testing data for {model_name}...")
(X_test, y_test, imagePaths_test) = getDataAndImagePaths(model_name, split_type = "test")
print("[INFO] data loaded...")

[INFO] loading training data for Cassava_Dataset_MobileNetv2_pretrained...
[INFO] splitting the datasets into train and validation...
[INFO] loading testing data for Cassava_Dataset_MobileNetv2_pretrained...
[INFO] data loaded...


In [5]:
# convert the labels from integers to one-hot-encoded vectors
lb = LabelBinarizer()
Y_train = lb.fit_transform(y_train)
Y_val = lb.transform(y_val)
Y_test = lb.transform(y_test)

In [6]:
# shape of features
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print()

# shape of labels
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print()

# shape of labels after encoding
print(Y_train.shape)
print(Y_val.shape)
print(Y_test.shape)

(10486, 294)
(4494, 294)
(6417, 294)

(10486,)
(4494,)
(6417,)

(10486, 5)
(4494, 5)
(6417, 5)


### Neural Network Classifier

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [8]:
input_shape = X_train.shape[1]
print(input_shape)

294


In [22]:
model = Sequential()
model.add(Dense(512, input_shape = (input_shape,), activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(5, activation="softmax"))

In [23]:
EPOCHS = 40
BATCH_SIZE = 16

model.compile(loss = "categorical_crossentropy", optimizer = Adam(learning_rate = 0.001), metrics = ["accuracy"])

In [24]:
# Reduce learning rate when there is a change lesser than <min_delta> in <val_accuracy> for more than <patience> epochs
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 mode = 'max',
                                                 min_delta = 0.01,
                                                 patience = 3,
                                                 factor = 0.25,
                                                 verbose = 1,
                                                 cooldown = 0,
                                                 min_lr = 0.0001)

# Stop the training process when there is a change lesser than <min_delta> in <val_accuracy> for more than <patience> epochs
early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 mode = 'max',
                                                 min_delta = 0.005,
                                                 patience = 10,
                                                 verbose = 1,
                                                 restore_best_weights = True)

# train the neural network
his_fe = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs = EPOCHS, 
                           batch_size = BATCH_SIZE, callbacks = [early_stopper, reduce_lr])

Epoch 1/40
656/656 [==============================] - 6s 7ms/step - loss: 0.9526 - accuracy: 0.6893 - val_loss: 0.9013 - val_accuracy: 0.6947 - lr: 0.0010
Epoch 2/40
656/656 [==============================] - 5s 7ms/step - loss: 0.7439 - accuracy: 0.7295 - val_loss: 0.8755 - val_accuracy: 0.6951 - lr: 0.0010
Epoch 3/40
656/656 [==============================] - 5s 7ms/step - loss: 0.7114 - accuracy: 0.7352 - val_loss: 0.8737 - val_accuracy: 0.6967 - lr: 0.0010
Epoch 4/40
656/656 [==============================] - 5s 7ms/step - loss: 0.6940 - accuracy: 0.7391 - val_loss: 0.8342 - val_accuracy: 0.7074 - lr: 0.0010
Epoch 5/40
656/656 [==============================] - 5s 7ms/step - loss: 0.6913 - accuracy: 0.7397 - val_loss: 0.8224 - val_accuracy: 0.7096 - lr: 0.0010
Epoch 6/40
656/656 [==============================] - 5s 7ms/step - loss: 0.6784 - accuracy: 0.7451 - val_loss: 0.8438 - val_accuracy: 0.7027 - lr: 0.0010
Epoch 7/40
651/656 [============================>.] - ETA: 0s - loss: 

In [25]:
score = model.evaluate(X_val, Y_val, verbose = 1)
print('Test accuracy:', score[1])

141/141 [==============================] - 0s 2ms/step - loss: 0.8074 - accuracy: 0.7176
Test accuracy: 0.7176234722137451


In [17]:
predy = model.predict(X_test)
pred = np.argmax(predy, axis = 1)
ground = np.argmax(Y_test, axis = 1)
print(classification_report(ground, pred))

201/201 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.35      0.25      0.29       326
           1       0.52      0.56      0.54       656
           2       0.54      0.27      0.36       715
           3       0.82      0.92      0.87      3947
           4       0.46      0.43      0.44       773

    accuracy                           0.71      6417
   macro avg       0.54      0.48      0.50      6417
weighted avg       0.69      0.71      0.70      6417



In [18]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [19]:
clf_SV = SVC(kernel = 'linear', probability = True)
clf_SV = clf_SV.fit(X_train, np.ravel(y_train))
y_pred = clf_SV.predict(X_test)

print("Accuracy on training set: {:.5f}".format(clf_SV.score(X_train, y_train)))
print("Accuracy on test set:     {:.5f}".format(clf_SV.score(X_test, y_test)))

Accuracy on training set: 0.80097
Accuracy on test set:     0.70983


In [39]:
clf_KNN = KNeighborsClassifier(n_neighbors = 60)
clf_KNN = clf_KNN.fit(X_train, np.ravel(y_train))
y_pred = clf_KNN.predict(X_test)

print("Accuracy on training set: {:.5f}".format(clf_KNN.score(X_train, y_train)))
print("Accuracy on test set:     {:.5f}".format(clf_KNN.score(X_test, y_test)))

Accuracy on training set: 0.76140
Accuracy on test set:     0.70157


In [45]:
clf_DT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, splitter = 'best')
clf_DT = clf_DT.fit(X_train, np.ravel(y_train))
y_pred = clf_DT.predict(X_test)

print("Accuracy on training set: {:.5f}".format(clf_DT.score(X_train, y_train)))
print("Accuracy on test set:     {:.5f}".format(clf_DT.score(X_test, y_test)))

Accuracy on training set: 0.71390
Accuracy on test set:     0.67695


In [54]:
clf_RF = RandomForestClassifier(max_depth = 7, random_state = 20)
clf_RF = clf_RF.fit(X_train, np.ravel(y_train))
y_pred = clf_RF.predict(X_test)

print("Accuracy on training set: {:.5f}".format(clf_RF.score(X_train, y_train)))
print("Accuracy on test set:     {:.5f}".format(clf_RF.score(X_test, y_test)))

Accuracy on training set: 0.78915
Accuracy on test set:     0.70890
